Задание 1.   
Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. 
Пункт отправления и пункт назначения должны передаваться в качестве параметров. 
Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию)

In [1]:
import requests
import pprint
from datetime import datetime, timedelta
import calendar


In [2]:
def getjson_aviasales(link, params):
    req = requests.get(link, params=params)
    return req.json()

def print_aviasales(params=['Дата вылета', 'Отправление', 'Назначение', 'Cтоимость', 'Продавец']):
    for spam in params:
        print(f"{spam:12.12}", sep='\t',  end=' |')
    print("", sep='\n')
    
origin =  'Москва'
destination = 'Санкт-Петербург'
search_params =  {
    'q': f'из {origin} в {destination}'
}#описание пути
airports = getjson_aviasales("https://www.travelpayouts.com/widgets_suggest_params", search_params )
#pprint.pprint(airports)

depart_date =  datetime.strptime('2019-09-14', '%Y-%m-%d')

flight_params = {
    'origin': airports['origin']['iata'],
    'destination': airports['destination']['iata'],
    'one_way': 'false',
    'depart_date': f"{depart_date:%Y-%m-%d}" #дата отправления
}
prices = getjson_aviasales("http://min-prices.aviasales.ru/calendar_preload", flight_params)
#pprint.pprint(flight_params)
#pprint.pprint(prices)
print_aviasales()
current_depart_date_prices = prices['current_depart_date_prices']
for spam in range(min(len(current_depart_date_prices), 10)):
    params = [current_depart_date_prices[spam]['depart_date'], current_depart_date_prices[spam]['origin'],
              current_depart_date_prices[spam]['destination'], current_depart_date_prices[spam]['value'], 
              current_depart_date_prices[spam]['gate']]
    print_aviasales(params)

Дата вылета  |Отправление  |Назначение   |Cтоимость    |Продавец     |
2019-09-14   |MOW          |LED          |      4180.0 |Aviakassa    |
2019-09-14   |MOW          |LED          |      4580.0 |Aviakassa    |
2019-09-14   |MOW          |LED          |      4180.0 |Aviakassa    |
2019-09-14   |MOW          |LED          |      5480.0 |Aviakassa    |
2019-09-14   |MOW          |LED          |      4400.0 |Pobeda       |
2019-09-14   |MOW          |LED          |      4400.0 |Pobeda       |
2019-09-14   |MOW          |LED          |      4400.0 |Pobeda       |
2019-09-14   |MOW          |LED          |      4400.0 |Pobeda       |
2019-09-14   |MOW          |LED          |      5300.0 |Pobeda       |
2019-09-14   |MOW          |LED          |      4180.0 |Aviakassa    |


Задание 2.     
В приложении парсинга википедии получить первую ссылку на другую страницу и вывести все значимые слова из неё. Результат записать в файл в форматированном виде
Задание 3.*Научить приложение определять количество ссылок в статье. Спарсить каждую ссылку и результаты записать в отдельные файлы.

In [3]:
import collections
import requests
import re
import urllib.parse
import json

# возвращает html код статьи
def return_wiki_html(topic):
    wiki_request = requests.get(f'https://ru.wikipedia.org/wiki/{topic.capitalize()}')
    return wiki_request.text

# возвращает 10 часто используемых слов статьи 
def return_words(topic):
    wiki_html = return_wiki_html(topic)
    words = re.findall('[а-яА-Я]{3,}', wiki_html)
    words_counter = collections.Counter()
    for word in words:
        words_counter[word] += 1
    return words_counter.most_common(10)

# печать 10 часто используемых слов статьи 
def print_words(topic):
    words = return_words(topic)
    for word in words:
        print(f'Слово {word[0]} встречается {word[1]} раз')
        
# возвращает список ссылок из статьи на другие темы        
def return_urls(topic):
    wiki_html = return_wiki_html(topic)
    regex = 'href=\"\/wiki\/(.+?)".+?'
    return list(map(lambda x: "https://ru.wikipedia.org/wiki/" + urllib.parse.unquote(x).capitalize(), 
                    re.findall(regex , wiki_html)))

# возвращает список тем, на которые есть ссылки в статье    
def return_topics(topic):
    wiki_html = return_wiki_html(topic)
    regex = 'href=\"\/wiki\/(.+?)".+?'
    return list(map(lambda x: urllib.parse.unquote(x).capitalize(), re.findall(regex , wiki_html)))

#сохраняет статистику слов в файл по указанной теме
def save_in_file(topic):    
    with open(topic.capitalize()+".txt", 'w') as file:
         json.dump(return_words(topic), file, ensure_ascii=False)
    print(f'Файл {topic}.txt сохранен')    

In [4]:
MAIN_TOPIC = 'Общая теория систем'
MAX_COUNT_TOPICS = 5 #ограничиваем список тем для удобства демонстрации
print(f'{MAX_COUNT_TOPICS} cсылок на другие темы в теме {MAIN_TOPIC}')
for url in return_urls(MAIN_TOPIC)[:MAX_COUNT_TOPICS]:
    print(url)  

5 cсылок на другие темы в теме Общая теория систем
https://ru.wikipedia.org/wiki/Система
https://ru.wikipedia.org/wiki/Системный_подход
https://ru.wikipedia.org/wiki/Берталанфи,_людвиг_фон
https://ru.wikipedia.org/wiki/Изоморфизм
https://ru.wikipedia.org/wiki/Кибернетика


In [5]:
print(f'Часто используемые слова в теме {MAIN_TOPIC}')
print_words(MAIN_TOPIC)

Часто используемые слова в теме Общая теория систем
Слово систем встречается 253 раз
Слово теории встречается 101 раз
Слово исследования встречается 89 раз
Слово теория встречается 83 раз
Слово Наука встречается 71 раз
Слово как встречается 58 раз
Слово Общая встречается 54 раз
Слово общей встречается 54 раз
Слово Берталанфи встречается 53 раз
Слово Системные встречается 53 раз


In [6]:
print(f'{MAX_COUNT_TOPICS} тем в статье {MAIN_TOPIC}')
return_topics(MAIN_TOPIC)[:MAX_COUNT_TOPICS]

5 тем в статье Общая теория систем


['Система',
 'Системный_подход',
 'Берталанфи,_людвиг_фон',
 'Изоморфизм',
 'Кибернетика']

In [7]:
for topic in return_topics(MAIN_TOPIC)[:MAX_COUNT_TOPICS]:
    save_in_file(topic)

Файл Система.txt сохранен
Файл Системный_подход.txt сохранен
Файл Берталанфи,_людвиг_фон.txt сохранен
Файл Изоморфизм.txt сохранен
Файл Кибернетика.txt сохранен
